In [1]:
import wrangle
import pyspark
import pyspark.ml
from pyspark.sql.functions import *

spark = pyspark.sql.SparkSession.builder.getOrCreate()

In [2]:
spark.version

'2.4.6'

In [3]:
df = wrangle.wrangle_311(spark)

[wrangle.py] reading case.csv
[wrangle.py] handling data types
[wrangle.py] parsing dates
[wrangle.py] adding features
[wrangle.py] joining departments


In [4]:
train, test = df.randomSplit([0.8, 0.2], seed=42)

In [6]:
# train.groupBy('case_age').count().show()

In [7]:
rf = pyspark.ml.feature.RFormula(
        formula='case_late ~ department + council_district')\
        .fit(train)

In [8]:
train_input = rf.transform(train)
train_input.show(1, vertical=True, truncate=False)

-RECORD 0----------------------------------------------------
 case_id              | 1014127332                           
 case_opened_date     | 2018-01-01 00:42:00                  
 case_closed_date     | 2018-01-01 12:29:00                  
 case_due_date        | 2020-09-26 00:42:00                  
 case_late            | false                                
 num_days_late        | -998.5087616000001                   
 case_closed          | true                                 
 service_request_type | Stray Animal                         
 SLA_days             | 999.0                                
 case_status          | Closed                               
 source_id            | svcCRMLS                             
 request_address      | 2315  EL PASO ST, San Antonio, 78207 
 council_district     | 005                                  
 num_weeks_late       | -142.6441088                         
 zipcode              | 78207                                
 case_ag

# Classification Models

In [9]:
train.dtypes

[('case_id', 'int'),
 ('case_opened_date', 'timestamp'),
 ('case_closed_date', 'timestamp'),
 ('case_due_date', 'timestamp'),
 ('case_late', 'boolean'),
 ('num_days_late', 'double'),
 ('case_closed', 'boolean'),
 ('service_request_type', 'string'),
 ('SLA_days', 'double'),
 ('case_status', 'string'),
 ('source_id', 'string'),
 ('request_address', 'string'),
 ('council_district', 'string'),
 ('num_weeks_late', 'double'),
 ('zipcode', 'string'),
 ('case_age', 'int'),
 ('days_to_closed', 'int'),
 ('case_lifetime', 'int'),
 ('department', 'string'),
 ('dept_subject_to_SLA', 'boolean')]

In [10]:
clas_model = pyspark.ml.classification.\
    LogisticRegression()
clas_model_fit = clas_model.fit(train_input)

clas_model_fit.summary.areaUnderROC

0.6372537251671417

In [11]:
evaluator = pyspark.ml.evaluation.BinaryClassificationEvaluator()
test_auc = evaluator.evaluate(
                    clas_model_fit.transform(rf.transform(test)))
test_auc

0.6328110531673008

In [12]:
clas_model = pyspark.ml.classification\
    .DecisionTreeClassifier(maxDepth=5)
clas_model_fit = clas_model.fit(train_input)

evaluator = pyspark.ml.evaluation.BinaryClassificationEvaluator()
test_auc = evaluator.evaluate(
                    clas_model_fit.transform(rf.transform(test)))
test_auc

0.49075879297284075

In [13]:
clas_model = pyspark.ml.classification\
    .RandomForestClassifier(numTrees=6)
clas_model_fit = clas_model.fit(train_input)

evaluator = pyspark.ml.evaluation.BinaryClassificationEvaluator()
test_auc = evaluator.evaluate(
                    clas_model_fit.transform(rf.transform(test)))
test_auc

0.6072719574269416

In [14]:
clas_model = pyspark.ml.classification.NaiveBayes()
clas_model_fit = clas_model.fit(train_input)

evaluator = pyspark.ml.evaluation.BinaryClassificationEvaluator()
test_auc = evaluator.evaluate(
                    clas_model_fit.transform(rf.transform(test)))
test_auc

0.5038396786125383

# Regression Models

In [32]:
train = train.dropna()

In [39]:
rf = pyspark.ml.feature.RFormula(
        formula='num_days_late ~ days_to_closed')\
        .fit(train)

train_input = rf.transform(train)
train_input.show(1, vertical=True, truncate=False)

-RECORD 0----------------------------------------------------
 case_id              | 1014127332                           
 case_opened_date     | 2018-01-01 00:42:00                  
 case_closed_date     | 2018-01-01 12:29:00                  
 case_due_date        | 2020-09-26 00:42:00                  
 case_late            | false                                
 num_days_late        | -998.5087616000001                   
 case_closed          | true                                 
 service_request_type | Stray Animal                         
 SLA_days             | 999.0                                
 case_status          | Closed                               
 source_id            | svcCRMLS                             
 request_address      | 2315  EL PASO ST, San Antonio, 78207 
 council_district     | 005                                  
 num_weeks_late       | -142.6441088                         
 zipcode              | 78207                                
 case_ag

In [53]:
enumerate(df.select('department').distinct())

In [40]:
lr = pyspark.ml.regression.LinearRegression()
lr_fit = lr.fit(train_input)
test_input = rf.transform(test)

evaluator = pyspark.ml.evaluation.RegressionEvaluator()
rmse = evaluator.evaluate(lr_fit.transform(test_input))
rmse

Py4JJavaError: An error occurred while calling o1773.evaluate.
: org.apache.spark.SparkException: Job aborted due to stage failure: Task 3 in stage 321.0 failed 1 times, most recent failure: Lost task 3.0 in stage 321.0 (TID 928, localhost, executor driver): org.apache.spark.SparkException: Failed to execute user defined function($anonfun$4: (struct<days_to_closed_double_RFormula_36b926f51133:double>) => struct<type:tinyint,size:int,indices:array<int>,values:array<double>>)
	at org.apache.spark.sql.catalyst.expressions.GeneratedClass$GeneratedIteratorForCodegenStage2.processNext(Unknown Source)
	at org.apache.spark.sql.execution.BufferedRowIterator.hasNext(BufferedRowIterator.java:43)
	at org.apache.spark.sql.execution.WholeStageCodegenExec$$anonfun$13$$anon$1.hasNext(WholeStageCodegenExec.scala:636)
	at scala.collection.Iterator$$anon$11.hasNext(Iterator.scala:409)
	at scala.collection.Iterator$$anon$11.hasNext(Iterator.scala:409)
	at scala.collection.Iterator$$anon$11.hasNext(Iterato

In [ ]:
lr = pyspark.ml.regression.DecisionTreeRegressor(maxDepth=5)
lr_fit = lr.fit(train_input)
test_input = rf.transform(test)

evaluator = pyspark.ml.evaluation.RegressionEvaluator()
rmse = evaluator.evaluate(lr_fit.transform(test_input))
rmse

In [ ]:
lr = pyspark.ml.regression.RandomForestRegressor(numTrees=12)
lr_fit = lr.fit(train_input)
test_input = rf.transform(test)

evaluator = pyspark.ml.evaluation.RegressionEvaluator()
rmse = evaluator.evaluate(lr_fit.transform(test_input))
rmse